In [ ]:
import subprocess
import os
import sys
from transformers import Wav2Vec2Processor, HubertForCTC
import torch
import torchaudio
from datasets import load_dataset
from IPython.display import Audio
import re

In [ ]:
# Extract audio from video

# def convert_video_to_audio_ffmpeg(video_file, output_ext="mp3"):
#     filename, ext = os.path.splitext(video_file)
#     subprocess.call(["ffmpeg", "-y", "-i", video_file, f"{filename}.{output_ext}"], 
#                     stdout=subprocess.DEVNULL,
#                     stderr=subprocess.STDOUT)
#convert_video_to_audio_ffmpeg('/home/delta/Downloads/youtube_tutorial_automation.mp4')

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft").to('cuda') #change to 'cpu' if no Nvidia graphic card

In [ ]:
audio_file = '/home/delta/Downloads/youtube_tutorial_automation.mp3'

In [ ]:
frame_offset, num_frames = 0, 16000*60*4  # num_frames depends on GPU size
number_of_frames = torchaudio.load(audio_file)[0].shape[1]
number_of_chunks = int(number_of_frames/num_frames)
remaining = number_of_frames % num_frames
transcriptions = ''
for i in range(number_of_chunks):
    data_waveform, rate_of_sample = torchaudio.load(audio_file,frame_offset=num_frames*i, num_frames=num_frames)
    transform = torchaudio.transforms.Resample(rate_of_sample, 16000)
    data_waveform = transform(data_waveform)
    inputs = processor(data_waveform[0], sampling_rate=16000, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs.to('cuda')).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    transcription = transcription[0].lower()
    transcriptions += transcription

data_waveform, rate_of_sample = torchaudio.load(audio_file,frame_offset=num_frames*(i+1), num_frames=remaining)
transform = torchaudio.transforms.Resample(rate_of_sample, 16000)
data_waveform = transform(data_waveform)
inputs = processor(data_waveform[0], sampling_rate=16000, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs.to('cuda')).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
transcription = transcription[0].lower()
transcriptions += transcription
transcriptions = re.sub(' +', ' ', transcriptions)
print(transcriptions)